<a href="https://colab.research.google.com/github/rkb-1/mrca/blob/master/UR5_Forward_Kinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forward Kinematic Analysis of UR5 6R robot arm

![picture](https://drive.google.com/uc?export=view&id=1YpDjtZiImtlOFIHhDU00im2a4f9rykjC)
Source: Modern Robotics, 2017 

**Universal Robots’ UR5 6R robot arm**
It consists of 6 revolute joints that connect the links in a serial chain. The above figure shows the screw axes $\mathbf S_1 \dots \mathbf S_6$ when the robot is at its zero position.

Dimensions : $W_1$= 109 mm , $W_2$ = 82 mm, $L_1$ = 425 mm, $L_2$ = 392 mm, $H_1$ = 89 mm,
$H_2$ = 95 mm

In [ ]:
# Import required libraries
import numpy as np
from sympy import Matrix, symbols, Identity, cos, sin, zeros, simplify

## Spatial Representation of Screw axes


In [ ]:
L_1, L_2, W_1, W_2, H_1, H_2 = symbols("L_1 L_2 W_1 W_2 H_1 H_2", real=1)  # link lenghts

In [ ]:
s1_hat = Matrix([0,0,1])
s1_o = Matrix([0,0,0])
display(s1_hat,s1_o)


In [ ]:
s2_hat = Matrix([0,1,0])
s2_o = Matrix([0,0,H_1])
display(s2_hat,s2_o)

In [ ]:
s3_hat = Matrix([0,1,0])
s3_o = Matrix([L_1,0,H_1])
display(s3_hat,s3_o)

In [ ]:
s4_hat = Matrix([0,1,0])
s4_o = Matrix([L_1+L_2,0,H_1])
display(s4_hat,s4_o)

In [ ]:
s5_hat = Matrix([0,0,-1])
s5_o = Matrix([L_1+L_2,W_1,H_1])
display(s5_hat,s5_o)

In [ ]:
s6_hat = Matrix([0,1,0])
s6_o = Matrix([L_1+L_2,W_1+W_2,H_1-H_2])
display(s6_hat,s6_o)

In [ ]:
S1 = Matrix([s1_hat, s1_o.cross(s1_hat)])
S2 = Matrix([s2_hat, s2_o.cross(s2_hat)])
S3 = Matrix([s3_hat, s3_o.cross(s3_hat)])
S4 = Matrix([s4_hat, s4_o.cross(s4_hat)])
S5 = Matrix([s5_hat, s5_o.cross(s5_hat)])
S6 = Matrix([s6_hat, s6_o.cross(s6_hat)])
print("S1:")
display(S1)
print("S2:")
display(S2)
print("S3:")
display(S3)
print("S4:")
display(S4)
print("S5:")
display(S5)
print("S6:")
display(S6)
S = S1.row_join(S2).row_join(S3).row_join(S4).row_join(S5).row_join(S6) 
print("Screw axes (arranged column wise for the joints) :\n")
display(S)

## Reference configuration of the end effector at zero position
A general configuration is given by a rotation matrix $\mathbf{R}$ and position vector $\mathbf{p}$ as follows.
\begin{equation}
        \mathbf{T} = \begin{bmatrix}
                \mathbf{R} & \textbf{p} \\ \mathbf{0} & 1
            \end{bmatrix}
    \end{equation}



In [ ]:
def rot_axisz(theta):
    """Returns a rotation matrix for a rotation of theta (in radians) about
    the 3-axis.
    [...]
    """
    lil = ((cos(theta), -sin(theta), 0),
           (sin(theta),  cos(theta), 0),
           (       0  ,       0   ,  1))
    return Matrix(lil)
def rot_axisx(theta):
    """Returns a rotation matrix for a rotation of theta (in radians) about
    the 3-axis.
    [...]
    """
    lil = ((1,       0   ,      0     ),
           (0, cos(theta), -sin(theta)),
           (0, sin(theta),  cos(theta)))
    return Matrix(lil)
def rot_axisy(theta):
    """Returns a rotation matrix for a rotation of theta (in radians) about
    the 3-axis.
    [...]
    """
    lil = (( cos(theta), 0, sin(theta)),
           (      0    , 1,     0     ),
           (-sin(theta), 0, cos(theta)))
    return Matrix(lil)

In [ ]:
p_0_ee = Matrix([ L_1 + L_2, W_1 + W_2 , H_1 - H_2 ])
R_0_ee = Matrix(rot_axisy(np.pi)*rot_axisx(-np.pi/2)) ## R_0_ee = R_y*R_x
T_0_ee = R_0_ee.row_join(p_0_ee).col_join(Matrix([0, 0, 0, 1]).T)
print("Zero configuration of end-effector pose: \n")
display(T_0_ee)

## Product of Exponential definition

Let $ \mathbf S = (\boldsymbol \xi, \boldsymbol \eta) $ denote the screw coordinates. 
The matrix exponential is defined as $\exp: [\mathbf S] \theta \in se(3) \to \mathbf T \in SE(3)$. 
 
If $||{\boldsymbol \xi}|| = 1$ then for any distance $\theta \in \mathbb{R}$ traveled along the screw axis or any angle $\theta \in \mathbb{R}$ rotated about the screw axis,
\begin{equation}
 \mathbf T = 
  \exp{[\mathbf S] \theta} 
  = \left[
\begin{matrix}
\exp{[\boldsymbol \xi] \theta} & (\mathbf I \theta + (1-\cos \theta)[\boldsymbol \xi] + (\theta - \sin \theta)[\boldsymbol \xi]^2) \boldsymbol \eta\\
\mathbf 0 & 1
\end{matrix}
 \right] \in SE(3)
 \end{equation}
where, 
\begin{equation}
\mathbf R = \exp{[\boldsymbol \xi] \theta} = \mathbf I + \sin \theta[\boldsymbol \xi] + (1 - \cos \theta)[\boldsymbol \xi]^2 \in SO(3) 
\end{equation}
and
\begin{equation}
\mathbf p = (\mathbf I \theta + (1-\cos \theta)[\boldsymbol \xi] + (\theta - \sin \theta)[\boldsymbol \xi]^2) \boldsymbol \eta \in \mathbb R^3 
\end{equation}
If $||{\boldsymbol \xi}|| = 0$ and $||{\boldsymbol \eta}|| = 1$ (prismatic joints), then
 \begin{equation}
  \mathbf T = 
  \exp{[\mathbf S] \theta} 
  = \left[
\begin{matrix}
\mathbf I &  \boldsymbol \eta \theta\\
\mathbf 0 & 1
\end{matrix}
 \right] \in SE(3) 
 \end{equation}




### The product of exponential (POE) formula is given by
\begin{equation}
  {}^0\mathbf T_n(\mathbf q) = \exp([\mathbf S_1]q_1) \exp([\mathbf S_2]q_2) \ldots \exp([\mathbf S_n]q_n) {}^0\mathbf T_n(\mathbf 0) 
\end{equation}


In [ ]:
def SE3Exp(XX, t):
        """compute exponential mapping for SE(3).
        """
        X = XX.T
        xi = Matrix(X[0:3])
        eta = Matrix(X[3:6])
        xihat = Matrix([[0, -xi[2], xi[1]],
                        [xi[2], 0, -xi[0]],
                        [-xi[1], xi[0], 0]])
        R = Matrix(Identity(3)) + sin(t)*xihat + (1-cos(t))*(xihat*xihat)
        p = Matrix(Identity(3)*t + (1-cos(t))*xihat + (t-sin(t))*(xihat*xihat))*eta
        T = R.row_join(p).col_join(Matrix([0, 0, 0, 1]).T)
        return T
print("Matrix exponential for screw axis 3 : \n")
display(SE3Exp(S3,symbols("x")))#example

## Forward Kinematics Using POE formula
### Define the generalised coordinates for the system, $q_1 \dots q_6$

In [ ]:
q1, q2, q3, q4, q5, q6 = symbols("q1 q2 q3 q4 q5 q6", real=1)  # joint positions
T_0_q = simplify(SE3Exp(S1,q1) * SE3Exp(S2,q2) * SE3Exp(S3,q3) * 
                 SE3Exp(S4,q4) * SE3Exp(S5,q5) * SE3Exp(S6,q6) * T_0_ee)

In [ ]:
display(T_0_q)

### Substitute with numerical values
$q1$ = 0
$q2$ = -pi/2
$q3$ = 0
$q4$ = 0
$q5$ = pi/2
$q6$ = 0
$W_1$ = 0.109 
$W_2$ = 0.082 
$L_1$ = 0.425 
$L_2$ = 0.392 
$H_1$ = 0.089
$H_2$ = 0.095

In [ ]:
fkin= T_0_q.evalf(4, subs={q1:0, q2:-np.pi/2, q3:0, q4:0, q5:np.pi/2, q6:0,
                          W_1: 0.109, W_2: 0.082,
                          L_1: 0.425, L_2: 0.392,
                          H_1: 0.089, H_2: 0.095})
print("Forward kinematics for the input joint angles: \n")
display(fkin)